In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

In [2]:
warnings.filterwarnings(action = 'ignore')

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string


2023-05-03 00:20:54.368261: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 00:20:54.416872: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 00:20:54.417799: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 00:20:55.266314: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-03 00:20:56.284526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

In [4]:
pip install -U spacy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

In [6]:
dev=pd.read_json('dev/EXIST2023_dev.json')

In [7]:
dev_Yes=pd.read_json('dev/EXIST2023_dev.json')
dev_Yes=dev_Yes.transpose()
dev_Yes=dev_Yes.loc[dev_Yes['lang'].apply(lambda x:x=='en')]
dev_Yes=dev_Yes.loc[dev_Yes['labels_task1'].apply(lambda x:x.count('YES')>3)]
dev_Yes['label1']='YES'
dev_Yes

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400002,400002,en,@BBCWomansHour @LabWomenDec @EverydaySexism Sh...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
400003,400003,en,#everydaysexism Some man moving my suitcase in...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
400008,400008,en,@ReproRights @AbortionStories Getting Twitter ...,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
400011,400011,en,@esjayXX @EcuadorianMum @monsalore They so rem...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
400013,400013,en,One of the depressing things about #NotAllMen ...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
400485,400485,en,@YesReallyAngel “Don’t wear a black bra with a...,6,"[Annotator_801, Annotator_182, Annotator_802, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, DIRECT, DIRECT, ...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, SEX...",DEV_EN,YES
400486,400486,en,""" get changed , you look like a prostitute . ""...",6,"[Annotator_801, Annotator_182, Annotator_802, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, NO, YES]","[DIRECT, REPORTED, REPORTED, REPORTED, -, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,YES
400487,400487,en,made this top and my mom gave me the “you look...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, NO]","[JUDGEMENTAL, REPORTED, REPORTED, REPORTED, RE...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",DEV_EN,YES
400488,400488,en,@DawnAnd91320913 I haven't seen anything that ...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,YES


In [8]:
dev_No=pd.read_json('dev/EXIST2023_dev.json')
dev_No=dev_No.transpose()
dev_No=dev_No.loc[dev_No['lang'].apply(lambda x:x=='en')]
dev_No=dev_No.loc[dev_No['labels_task1'].apply(lambda x:x.count('NO')>3)]
dev_No['label1']='NO'
dev_No

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400001,400001,en,"@Mike_Fabricant “You should smile more, love. ...",6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, YES, YES]","[-, -, -, -, REPORTED, DIRECT]","[[-], [-], [-], [-], [IDEOLOGICAL-INEQUALITY, ...",DEV_EN,NO
400004,400004,en,@KolHue @OliverJia1014 lol gamergate the go to...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, NO, NO, NO]","[DIRECT, -, -, -, -, -]","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",DEV_EN,NO
400005,400005,en,@ShelfStoriesGBL To me this has the same negat...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, YES, NO, NO]","[JUDGEMENTAL, -, -, REPORTED, -, -]","[[IDEOLOGICAL-INEQUALITY], [-], [-], [MISOGYNY...",DEV_EN,NO
400006,400006,en,@IrrelevantCmnt @jbo911 @BanButterfly @TheRigh...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, NO, NO, NO]","[DIRECT, -, -, -, -, -]","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",DEV_EN,NO
400007,400007,en,@cathymwafer @andrew_lilico Showing off? The m...,6,"[Annotator_785, Annotator_786, Annotator_787, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",DEV_EN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
400474,400474,en,#cryptosterone 👈All-new!Twice as many #men inv...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
400476,400476,en,@ScubaMySteve If you ask Cenk as a hypothetica...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
400477,400477,en,@snowflowergomi @minseulsoo @chimsmaze How wil...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
400481,400481,en,"Uh, they're allowed to? calling this out makes...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [9]:
devs = [dev_Yes, dev_No]
dev = pd.concat(devs)
dev.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,400002,en,@BBCWomansHour @LabWomenDec @EverydaySexism Sh...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
1,400003,en,#everydaysexism Some man moving my suitcase in...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
2,400008,en,@ReproRights @AbortionStories Getting Twitter ...,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
3,400011,en,@esjayXX @EcuadorianMum @monsalore They so rem...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
4,400013,en,One of the depressing things about #NotAllMen ...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
439,400474,en,#cryptosterone 👈All-new!Twice as many #men inv...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
440,400476,en,@ScubaMySteve If you ask Cenk as a hypothetica...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
441,400477,en,@snowflowergomi @minseulsoo @chimsmaze How wil...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
442,400481,en,"Uh, they're allowed to? calling this out makes...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [10]:
train_Yes=pd.read_json('training/EXIST2023_training.json')
train_Yes=train_Yes.transpose()
train_Yes=train_Yes.loc[train_Yes['lang'].apply(lambda x:x=='en')]
train_Yes=train_Yes.loc[train_Yes['labels_task1'].apply(lambda x:x.count('YES')>3)]
train_Yes['label1']='YES'
train_Yes

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200002,200002,en,Writing a uni essay in my local pub with a cof...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
200003,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
200006,200006,en,According to a customer I have plenty of time ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
200007,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
200011,200011,en,#EverydaySexism means women usually end up in ...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
203256,203256,en,idk why y’all bitches think having half your a...,6,"[Annotator_478, Annotator_479, Annotator_480, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, DIRECT, DIRECT, DIRECT, JUDGEMEN...","[[OBJECTIFICATION], [STEREOTYPING-DOMINANCE, S...",TRAIN_EN,YES
203257,203257,en,This has been a part of an experiment with @Wo...,6,"[Annotator_668, Annotator_669, Annotator_670, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, JUDGEMENTAL, DIRECT, J...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",TRAIN_EN,YES
203258,203258,en,"""Take me already"" ""Not yet. You gotta be ready...",6,"[Annotator_467, Annotator_468, Annotator_469, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, DIRECT, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [-], [SEXUAL-VIOLENCE...",TRAIN_EN,YES
203259,203259,en,@clintneedcoffee why do you look like a whore?...,6,"[Annotator_674, Annotator_675, Annotator_676, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, JUDGEMENTAL, ...","[[OBJECTIFICATION, SEXUAL-VIOLENCE, MISOGYNY-N...",TRAIN_EN,YES


In [11]:
train_No=pd.read_json('training/EXIST2023_training.json')
train_No=train_No.transpose()
train_No=train_No.loc[train_No['lang'].apply(lambda x:x=='en')]
train_No=train_No.loc[train_No['labels_task1'].apply(lambda x:x.count('NO')>3)]
train_No['label1']='NO'
train_No

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200008,200008,en,New to the shelves this week - looking forward...,6,"[Annotator_420, Annotator_296, Annotator_421, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY], [-],...",TRAIN_EN,NO
200010,200010,en,I guess that’s fairly normal for a Neanderthal...,6,"[Annotator_431, Annotator_432, Annotator_433, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, JUDGEMENTAL, -]","[[-], [-], [-], [-], [OBJECTIFICATION, SEXUAL-...",TRAIN_EN,NO
200023,200023,en,@JavedLSterritt He's really shit on games jour...,6,"[Annotator_496, Annotator_497, Annotator_498, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",TRAIN_EN,NO
200024,200024,en,"Low key, she was fr created because the Incred...",6,"[Annotator_502, Annotator_503, Annotator_504, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, JUDGEMENTAL]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",TRAIN_EN,NO
200025,200025,en,"One of the most important (and of course, unde...",6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_EN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
203180,203180,en,"@BFriedmanDC ""The other officers yell after Jo...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
203201,203201,en,@v1sig0th @PTSDGuts2 Get off your goddamn high...,6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
203202,203202,en,I hate the phrase “when you assume it makes an...,6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
203205,203205,en,To the guy who beeped at me after I'd put my h...,6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [12]:
frames = [train_Yes, train_No]
result = pd.concat(frames)
result.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,200002,en,Writing a uni essay in my local pub with a cof...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
1,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
2,200006,en,According to a customer I have plenty of time ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
3,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
4,200011,en,#EverydaySexism means women usually end up in ...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
2865,203180,en,"@BFriedmanDC ""The other officers yell after Jo...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
2866,203201,en,@v1sig0th @PTSDGuts2 Get off your goddamn high...,6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
2867,203202,en,I hate the phrase “when you assume it makes an...,6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
2868,203205,en,To the guy who beeped at me after I'd put my h...,6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [13]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [14]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string

In [15]:
gensim.__version__

'4.3.1'

In [16]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [17]:
wv = api.load('word2vec-google-news-300')

In [18]:
wv.save('training/vector.kv')

In [19]:
type(wv)

gensim.models.keyedvectors.KeyedVectors

In [20]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [21]:
import preprocessor as p

In [22]:

def preprocess_tweet(row):
    text = row['tweet']
    text = p.clean(text)
    return text

In [23]:
result['tweet'] = result.apply(preprocess_tweet, axis=1)
dev['tweet'] = dev.apply(preprocess_tweet, axis=1)

In [24]:
from gensim.parsing.preprocessing import remove_stopwords

In [25]:
def stopword_removal(row):
    text = row['tweet']
    text = remove_stopwords(text)
    
    return text

In [26]:
import re
result['tweet'] = result.apply(stopword_removal, axis=1)
result['tweet'] = result['tweet'].str.lower()
result.tweet = result.tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
result.tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r'@mention', '', x))
dev['tweet'] = dev.apply(stopword_removal, axis=1)
dev['tweet'] = dev['tweet'].str.lower()
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
dev.tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'@mention', '', x))

In [27]:
result.head()


,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200002,200002,en,writing uni essay local pub coffee random old ...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
200003,200003,en,i dont appreciate rides team member looked as...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
200006,200006,en,according customer i plenty time spent stirlin...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
200007,200007,en,so 'blokes' drink beer sorry aren't 'bloke' dr...,6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
200011,200011,en,means women usually end lower paid support wor...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES


In [28]:
dev.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400002,400002,en,she right push opposite direction - converting...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
400003,400003,en,some man moving suitcase overhead luggage stor...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
400008,400008,en,getting twitter harassed pro choice,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
400011,400011,en,they remind mgtow (men way) us men hate women ...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
400013,400013,en,one depressing things primarily rallying privi...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES


In [29]:
len(wv['apple'])

300

In [30]:
X_train=result['tweet']
X_test=dev['tweet']
 


In [31]:
dev

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400002,400002,en,she right push opposite direction - converting...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
400003,400003,en,some man moving suitcase overhead luggage stor...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
400008,400008,en,getting twitter harassed pro choice,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
400011,400011,en,they remind mgtow (men way) us men hate women ...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
400013,400013,en,one depressing things primarily rallying privi...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
400474,400474,en,all-newtwice invest means twice smart,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
400476,400476,en,if ask cenk hypothetical kind thinking justify...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
400477,400477,en,how dragged actions i showed i standing i told...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
400481,400481,en,uh they're allowed to calling makes look like ...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [32]:
y_test = dev.iloc[:, -1].values
y_test =pd.Series(y_test).map({'YES': 1, 'NO': 0})
y_train = result.iloc[:, -1].values
y_train =pd.Series(y_train).map({'YES': 1, 'NO': 0})

In [33]:
def tokenize_sent(sent):
    return [word for word in sent.split(" ")]

In [34]:
result['tweet'] = result['tweet'].apply(tokenize_sent)
result

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200002,200002,en,"[writing, uni, essay, local, pub, coffee, rand...",6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
200003,200003,en,"[, i, dont, appreciate, rides, team, member, l...",6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
200006,200006,en,"[according, customer, i, plenty, time, spent, ...",6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
200007,200007,en,"[so, 'blokes', drink, beer, sorry, aren't, 'bl...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
200011,200011,en,"[means, women, usually, end, lower, paid, supp...",6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
203180,203180,en,"[the, officers, yell, jones, repeatedly, 'slow...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
203201,203201,en,"[get, goddamn, high, horse, dude, k, easily, a...",6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
203202,203202,en,"[i, hate, phrase, assume, makes, ass, bc, true...",6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
203205,203205,en,"[to, guy, beeped, i'd, hazards, car, breaking,...",6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [35]:
dev['tweet'] = dev['tweet'].apply(tokenize_sent)
dev

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400002,400002,en,"[she, right, push, opposite, direction, -, con...",6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
400003,400003,en,"[some, man, moving, suitcase, overhead, luggag...",6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
400008,400008,en,"[getting, twitter, harassed, pro, choice, ]",6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
400011,400011,en,"[they, remind, mgtow, (men, way), us, men, hat...",6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
400013,400013,en,"[one, depressing, things, primarily, rallying,...",6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
400474,400474,en,"[all-newtwice, invest, , means, twice, smart]",6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
400476,400476,en,"[if, ask, cenk, hypothetical, kind, thinking, ...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
400477,400477,en,"[how, dragged, actions, i, showed, i, standing...",6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
400481,400481,en,"[uh, they're, allowed, to, calling, makes, loo...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [36]:
print(max(result['tweet'].apply(len)))
print(max(dev['tweet'].apply(len)))

51
36


In [37]:
def vectorize_tweet(tweet):
    vec = []
    for word in tweet:
        if word in wv.key_to_index:
            vec.append(wv[word])
    return vec

In [38]:
result['tweet'] = result['tweet'].apply(vectorize_tweet)


In [39]:
dev['tweet'] = dev['tweet'].apply(vectorize_tweet)

In [40]:
print(max(result['tweet'].apply(len)))
print(max(dev['tweet'].apply(len)))

40
33


In [59]:
import random
X_train = list(result['tweet'])
Y_train = list(result['label1'])

X_test = list(dev['tweet'])
Y_test = list(dev['label1'])

temp = list(zip(X_train, Y_train))
random.shuffle(temp)
X_train, Y_train = zip(*temp)
X_train = list(X_train)
Y_train = list(Y_train)

In [58]:
Y_train

['YES',
 'NO',
 'YES',
 'YES',
 'YES',
 'YES',
 'YES',
 'NO',
 'YES',
 'YES',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'NO',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'NO',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'YES',
 'YES',
 'YES',
 'YES',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'NO',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'YES',
 'NO',
 'YES',
 'YES',
 'YES',
 'NO',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'NO',
 'NO',
 'YES',
 'NO',
 'NO',
 'YES',
 'YES',
 'NO',
 'NO',
 'YES',
 'YES',
 'NO',
 'YE

In [42]:
def padto40(x):
    if len(x) < 40:
        for i in range(40 - len(x)):
            x.append([0]*300)
    return x

In [43]:
for i in range(len(X_train)):
    X_train[i] = padto40(X_train[i])

for i in range(len(X_test)):
    X_test[i] = padto40(X_test[i])

In [44]:
import tensorflow as tf

In [45]:
print(max(result['tweet'].apply(len)))
print(max(dev['tweet'].apply(len)))

40
40


In [46]:
print(min(result['tweet'].apply(len)))
print(min(dev['tweet'].apply(len)))

40
40


In [47]:
X_train_tensor = tf.convert_to_tensor(X_train)
X_test_tensor = tf.convert_to_tensor(X_test)

2023-05-03 00:22:15.004232: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 137760000 exceeds 10% of free system memory.


In [66]:
for i in range(len(Y_train)):
    if Y_train[i] == 'YES':
        Y_train[i] = [1, 0]
    else:
        Y_train[i] = [0, 1]

for i in range(len(Y_test)):
    if Y_test[i] == 'YES':
        Y_test[i] = [1, 0]
    else:
        Y_test[i] = [0, 1]

In [67]:
Y_train_tensor = tf.convert_to_tensor(Y_train)

In [68]:
Y_test_tensor = tf.convert_to_tensor(Y_test)


In [77]:
MLmodel = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(2, activation='softmax')
])
MLmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
MLmodel.build(input_shape=(None, 40, 300))
MLmodel.summary()

2023-05-03 00:35:36.537368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 00:35:36.538854: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 00:35:36.540040: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 300)              541200    
 l)                                                              
                                                                 
 dense_5 (Dense)             (None, 2)                 602       
                                                                 
Total params: 541,802
Trainable params: 541,802
Non-trainable params: 0
_________________________________________________________________


2023-05-03 00:35:36.742026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 00:35:36.743520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 00:35:36.744726: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
num_epochs = 20

MLmodel.fit(X_train_tensor, Y_train_tensor, epochs=num_epochs, validation_data=(X_test_tensor, Y_test_tensor))

Epoch 1/20


2023-05-03 00:35:41.227407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 00:35:41.229105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 00:35:41.230433: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

89/90 [============================>.] - ETA: 0s - loss: 0.6765 - accuracy: 0.6008

2023-05-03 00:35:47.496343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 00:35:47.497943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 00:35:47.499185: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

90/90 [==============================] - 7s 46ms/step - loss: 0.6767 - accuracy: 0.6007 - val_loss: 0.7007 - val_accuracy: 0.5631
Epoch 2/20
90/90 [==============================] - 3s 38ms/step - loss: 0.6611 - accuracy: 0.6017 - val_loss: 0.6858 - val_accuracy: 0.5698
Epoch 3/20
90/90 [==============================] - 3s 38ms/step - loss: 0.6411 - accuracy: 0.6321 - val_loss: 0.7037 - val_accuracy: 0.5360
Epoch 4/20
90/90 [==============================] - 3s 38ms/step - loss: 0.5994 - accuracy: 0.6760 - val_loss: 0.7420 - val_accuracy: 0.5541
Epoch 5/20
90/90 [==============================] - 3s 38ms/step - loss: 0.5544 - accuracy: 0.7136 - val_loss: 0.7888 - val_accuracy: 0.5428
Epoch 6/20
90/90 [==============================] - 3s 38ms/step - loss: 0.4994 - accuracy: 0.7463 - val_loss: 0.9100 - val_accuracy: 0.5586
Epoch 7/20
90/90 [==============================] - 4s 41ms/step - loss: 0.4220 - accuracy: 0.8010 - val_loss: 0.9780 - val_accuracy: 0.5270
Epoch 8/20
90/90 [======